# Amazon SageMaker + WhyLabs

In [1]:
import os
import json
import pandas as pd
from joblib import dump
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import boto3
from dotenv import dotenv_values
from utils import delete_model, delete_endpoint_config, delete_endpoint, is_endpoint_running

## AWS configuration

In [2]:
AWS_PROFILE_NAME = "mfa"
session = boto3.session.Session(profile_name=AWS_PROFILE_NAME)
AWS_REGION_NAME = session.region_name

In [3]:
sts = session.client("sts")
sm = session.client('sagemaker', region_name=AWS_REGION_NAME)
AWS_ACCOUNT_ID = sts.get_caller_identity().get("Account")
DOCKER_IMAGE_NAME = "whylabs-sagemaker"

## Download Iris dataset

In [4]:
!cd code/ && chmod +x download_iris.sh && ./download_iris.sh

  0%|                                               | 0.00/3.60k [00:00<?, ?B/s]
100%|██████████████████████████████████████| 3.60k/3.60k [00:00<00:00, 2.38MB/s]
Archive:  iris.zip
  inflating: Iris.csv                
  inflating: database.sqlite         


## Train data

In [5]:
data = pd.read_csv('code/dataset/iris.csv')
# Separating the independent variables from dependent variables
X = data.drop(['Id', 'Species'], axis=1)
y = data['Species']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [6]:
# Train a classifier
print("Train started...")
model = SVC()
model.fit(x_train, y_train)
print("Train ended...")
# Save the model
dump(model, 'code/model.joblib')
print("Model saved!")

Train started...
Train ended...
Model saved!


## Custom image building and pushing to ECR

In [7]:
os.system(f"./build_push.sh {DOCKER_IMAGE_NAME} {AWS_PROFILE_NAME}")

Image name whylabs-sagemaker
Profile name mfa
{
    "repositories": [
        {
            "repositoryArn": "arn:aws:ecr:us-east-1:377983720232:repository/whylabs-sagemaker",
            "registryId": "377983720232",
            "repositoryName": "whylabs-sagemaker",
            "repositoryUri": "377983720232.dkr.ecr.us-east-1.amazonaws.com/whylabs-sagemaker",
            "createdAt": "2021-09-08T17:32:14-05:00",
            "imageTagMutability": "MUTABLE",
            "imageScanningConfiguration": {
                "scanOnPush": false
            },
            "encryptionConfiguration": {
                "encryptionType": "AES256"
            }
        }
    ]
}
Login Succeeded


#1 [internal] load build definition from Dockerfile
#1 sha256:b8e7433dfee4f7ed8d6dc0fefb18dc20390190f6d52e521b7ccfe47ca756829e
#1 transferring dockerfile: 1.54kB 0.0s done
#1 DONE 0.0s

#2 [internal] load .dockerignore
#2 sha256:e73cd679c8fda7df9b86ac869ffaf2f99f7908427037a7d7b5e62b551cecbd40
#2 transferring context: 180B done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/ubuntu:18.04
#3 sha256:ae46bbb1b755529d0da663ca0256a22acd7c9fe21844946c149800baa67c4e4b
#3 ...

#4 [auth] library/ubuntu:pull token for registry-1.docker.io
#4 sha256:68ac9a20f42b3821532595ab49b2d680fc5e8ff3e66d06f8b759abd252125da6
#4 DONE 0.0s

#3 [internal] load metadata for docker.io/library/ubuntu:18.04
#3 sha256:ae46bbb1b755529d0da663ca0256a22acd7c9fe21844946c149800baa67c4e4b
#3 DONE 1.8s

#15 [ 1/11] FROM docker.io/library/ubuntu:18.04@sha256:9bc830af2bef73276515a29aa896eedfa7bdf4bdbc5c1063b4c457a4bbb8cd79
#15 sha256:d1750e31869fe5a60e2fad31896f5d8b06a6c26d3a20b7f5836401e641279689
#15 DONE 0.0s

The push refers to repository [377983720232.dkr.ecr.us-east-1.amazonaws.com/whylabs-sagemaker]
5f70bf18a086: Preparing
3b202542e03c: Preparing
0db6bc60cf67: Preparing
66d55ab066d7: Preparing
94df9b81d20b: Preparing
1d32553c20b9: Preparing
87953760e835: Preparing
42fafac3315e: Preparing
d20859943999: Preparing
cb8615e15e41: Preparing
6babb56be259: Preparing
d20859943999: Waiting
cb8615e15e41: Waiting
1d32553c20b9: Waiting
87953760e835: Waiting
42fafac3315e: Waiting
6babb56be259: Waiting
5f70bf18a086: Layer already exists
94df9b81d20b: Pushed
66d55ab066d7: Pushed
3b202542e03c: Pushed
1d32553c20b9: Pushed
d20859943999: Layer already exists
0db6bc60cf67: Pushed
6babb56be259: Layer already exists
cb8615e15e41: Layer already exists
42fafac3315e: Pushed
87953760e835: Pushed
latest: digest: sha256:9bc3ce4ea9d5b21b18c6d300c7d0d89239dbb51fabd3be9a7494ffd038901809 size: 2615


0

## Create SageMaker Endpoint

The steps to deploy a SageMaker model are:

1. Create a model
2. Create an endpoint configuration
3. Create a SageMaker endpoint

### 1. Model Creation

In [8]:
ECR_IMAGE_URI = f"{AWS_ACCOUNT_ID}.dkr.ecr.{AWS_REGION_NAME}.amazonaws.com/{DOCKER_IMAGE_NAME}:latest"
ENDPOINT_NAME = "whylabs-sagemaker"
EXECUTION_ROLE_ARN = f"arn:aws:iam::{AWS_ACCOUNT_ID}:role/SageMakerExecution"
INSTANCE_TYPE = "ml.m4.xlarge"

Load variables important for __WhyLabs configuration__ defined inside __.env file__ as dictionary. This values will be settled once the docker container is running within SageMaker.

In [9]:
# Load .env file as dictionary
environment = dotenv_values("code/.env")

In [10]:
# ECR image to be used
PRIMARY_CONTAINER = {
    'Image': ECR_IMAGE_URI, 
    "Environment": environment,
}

In [11]:
try:
    # Create sagemaker model
    r = sm.create_model(
        ModelName=ENDPOINT_NAME,
        ExecutionRoleArn=EXECUTION_ROLE_ARN,
        PrimaryContainer=PRIMARY_CONTAINER,
    )
    print("SageMaker model created.")
except Exception as e:
    print(e.response["Error"])

SageMaker model created.


### 2. Endpoint Config creation

In [12]:
ENDPOINT_CONFIG_NAME = ENDPOINT_NAME + '-config'

In [13]:
try:
    # create endpoint configuration
    _ = sm.create_endpoint_config(
        EndpointConfigName=ENDPOINT_CONFIG_NAME,
        ProductionVariants=[
            {
                'InstanceType': INSTANCE_TYPE,
                'InitialVariantWeight': 1,
                'InitialInstanceCount': 1,
                'ModelName': ENDPOINT_NAME,
                'VariantName': 'AllTraffic'
            }
        ]
    )
    print("Endpoint configuration created.")
except Exception as e:
    print(e.response["Error"])

Endpoint configuration created.


### 3. Endpoint creation

In [14]:
try:
    # create endpoint
    r = sm.create_endpoint(
        EndpointName=ENDPOINT_NAME,
        EndpointConfigName=ENDPOINT_CONFIG_NAME
    )
    print(f"Completed {ENDPOINT_NAME} model endpoint deployment !!!")
except Exception as e:
    print(e.response["Error"])

Completed whylabs-sagemaker model endpoint deployment !!!


## Test Endpoint 

In [15]:
# Payload for /invocations endpoint
payload = json.dumps({
    "sepal_length_cm": 5.1,
    "sepal_width_cm": 3.5,
    "petal_length_cm": 1.4,
    "petal_width_cm": 0.2
})

In [17]:
# Invoke the endpoint using
sg = session.client("runtime.sagemaker", region_name=AWS_REGION_NAME)
status = is_endpoint_running(ENDPOINT_NAME, AWS_PROFILE_NAME, AWS_REGION_NAME)
# Check if model was created successfully
if status == "InService":
    response = sg.invoke_endpoint(
        EndpointName=ENDPOINT_NAME,
        Body=payload,
        ContentType='application/json',
    )
    # Decode the response
    print(json.loads(response["Body"].read().decode("utf-8")))
else:
    print(f"Endpoint status is {status}.")

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (502) from primary with message "<html>
<head><title>502 Bad Gateway</title></head>
<body bgcolor="white">
<center><h1>502 Bad Gateway</h1></center>
<hr><center>nginx/1.14.0 (Ubuntu)</center>
</body>
</html>
". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/whylabs-sagemaker in account 377983720232 for more information.

## Delete AWS resources

In [ ]:
status = is_endpoint_running(ENDPOINT_NAME, AWS_PROFILE_NAME, AWS_REGION_NAME)

In [ ]:
if status == "InService":
    delete_model(sm, ENDPOINT_NAME)
    delete_endpoint_config(sm, ENDPOINT_CONFIG_NAME)
    delete_endpoint(sm, ENDPOINT_NAME)